In [9]:
import pandas as pd
import numpy as np
import json
from tqdm import tqdm
from joblib import Parallel, delayed
import pickle
import re
import multiprocessing



In [12]:
search1_all

,Unnamed: 0,form,date,cik,company,link,doc,keyword,serp,year,unique_id
0,0001803599-24-000156,10-Q,2024-10-04,1803599,Concentrix Corp,https://www.sec.gov/Archives/edgar/data/180359...,NOTE 11—PENSION AND EMPLOYEE BENEFITS PLANS: \...,"['2023, the Company contributed $22,945 and $6...",0,2024,0
1,0001552781-24-000563,DEF 14A,2024-10-04,21076,CLOROX CO /DE/,https://www.sec.gov/Archives/edgar/data/21076/...,Because the IRC limits the benefit value that ...,['Pension Plan (the Pension Plan) is a cash ba...,0,2024,1
2,0001552781-24-000563,DEF 14A,2024-10-04,21076,CLOROX CO /DE/,https://www.sec.gov/Archives/edgar/data/21076/...,(5) | The amounts reflect the aggregate change...,['2022 under the Pension Plan and the cash bal...,0,2024,2
3,0001552781-24-000563,DEF 14A,2024-10-04,21076,CLOROX CO /DE/,https://www.sec.gov/Archives/edgar/data/21076/...,(4) | The Pension Plan was frozen effective Ju...,"['Pension Plan was frozen effective June 30, 2...",0,2024,3
4,0001552781-24-000563,DEF 14A,2024-10-04,21076,CLOROX CO /DE/,https://www.sec.gov/Archives/edgar/data/21076/...,(5) | The cash balance restoration provision i...,"['2011, when the Pension Plan was frozen']",0,2024,4
...,...,...,...,...,...,...,...,...,...,...,...
7582,0000950136-01-500150,DEF 14A,2001-04-30,887921,REVLON INC /DE/,https://www.sec.gov/Archives/edgar/data/887921...,---------- (a) The normal form of benefit for ...,"['Retirement Plan ""frozen"" at the current leve...",0,2001,7582
7583,0000950130-01-501016,DEF 14A,2001-04-30,89261,EXX INC/NV/,https://www.sec.gov/Archives/edgar/data/89261/...,____________________ (1) The named individual ...,['defined benefit pension plan for salaried em...,0,2001,7583
7584,0000950152-00-002008,10-K,2000-03-23,899723,OM GROUP INC,https://www.sec.gov/Archives/edgar/data/899723...,"During 1998, the Company froze its salaried pe...","['1998, the Company froze its salaried pension...",0,2000,7584
7585,0000946275-00-000589,10-K,2000-12-29,1123276,FLORIDAFIRST BANCORP INC,https://www.sec.gov/Archives/edgar/data/112327...,"63 Capital at September 30, 2000 for consolid...","['1998, the Board of Directors froze benefit']",1,2000,7585


In [10]:
search1_all= pd.read_excel('data/search1_all_alternative.xlsx')
link = pd.read_csv('gvkey_cik_link.csv', encoding='latin1')
link_permno = pd.read_csv('cik_permno.csv', encoding='latin1')
link_permno[['cik', 'gvkey', 'LPERMNO','LPERMCO','LINKDT', 'LINKENDDT']]

C:\Users\TJR\AppData\Local\Temp\ipykernel_26764\4228950153.py:3: DtypeWarning: Columns (34) have mixed types. Specify dtype option on import or set low_memory=False.
  link_permno = pd.read_csv('cik_permno.csv', encoding='latin1')


,cik,gvkey,LPERMNO,LPERMCO,LINKDT,LINKENDDT
0,NaN,1000,25881,23369,1970-11-13,1978-06-30
1,723576.0,1001,10015,6398,1983-09-20,1986-07-31
2,1306124.0,1002,10023,22159,1972-12-14,1973-06-05
3,730052.0,1003,10031,6672,1983-12-07,1989-08-16
4,1750.0,1004,54594,20000,1972-04-24,E
...,...,...,...,...,...,...
32042,1892500.0,349994,23514,59438,2022-11-15,E
32043,1867325.0,350681,22205,58855,2021-10-22,2023-03-31
32044,1671502.0,351038,16161,55612,2021-10-29,E
32045,1944057.0,352262,23773,59507,2023-03-17,E


In [11]:
# use sequence as unique identifier
search1_all['unique_id'] = search1_all.index

df = pd.merge(search1_all, link, on='cik', how='left')

# create a new column, identify whether date is between link_start_date and link_end_date
df['date'] = pd.to_datetime(df['date'])
df['link_start_date'] = pd.to_datetime(df['link_start_date'], format='%Y-%m-%d', errors='coerce')
df['link_end_date'] = pd.to_datetime(df['link_end_date'], format='%Y-%m-%d', errors='coerce')
df['link_date'] = (df['date'] >= df['link_start_date']) & (df['date'] <= df['link_end_date'])

# for loop each unique id, 1. if there is more than one row, keep the ones with link_date = True 2. if there is only one row, keep it
def filter_link_date(df):
    if len(df) > 1 and df['link_date'].sum() > 0:
        return df[df['link_date']]
    else:
        return df
df = df.groupby('unique_id').apply(filter_link_date).reset_index(drop=True)
print(df.unique_id.nunique())
df.drop_duplicates(subset=['unique_id'], keep='first', inplace=True)

df = pd.merge(df, link_permno, on='cik', how='left')
df['date'] = pd.to_datetime(df['date'])
df['LINKDT'] = pd.to_datetime(df['LINKDT'], format='%Y-%m-%d', errors='coerce')
df['LINKENDDT'] = pd.to_datetime(df['LINKENDDT'], format='%Y-%m-%d', errors='coerce')
df['link_date_permno'] = (df['date'] >= df['LINKDT']) & (df['date'] <= df['LINKENDDT'])

def filter_link_date_permno(df):
    if len(df) > 1 and df['link_date_permno'].sum() > 0:
        return df[df['link_date_permno']]
    else:
        return df
    
df = df.groupby('unique_id').apply(filter_link_date_permno).reset_index(drop=True)
print(df.unique_id.nunique())
df.drop_duplicates(subset=['unique_id'], keep='first', inplace=True)

C:\Users\TJR\AppData\Local\Temp\ipykernel_26764\4198937823.py:18: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby('unique_id').apply(filter_link_date).reset_index(drop=True)


7587
7587


C:\Users\TJR\AppData\Local\Temp\ipykernel_26764\4198937823.py:34: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby('unique_id').apply(filter_link_date_permno).reset_index(drop=True)


In [13]:
df.to_excel('search1_all.xlsx', index=False)

In [ ]:
for year in range(2000,2021):
    df_year = df[df['year'] == year].drop(columns=['unique_id', 'link_start_date', 'link_end_date', 'link_date'])
    df_year.to_csv(f'search1_{year}.csv', index=False)